In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import pandas as pd

In [2]:
ua = UserAgent()
chrome = {'ua.agent':ua.chrome}
url = "https://www.1mg.com/categories/medical-devices/health-monitors/thermometers-167"
response = requests.get(url,headers=chrome)
soup = BeautifulSoup(response.text, 'lxml')

In [3]:
n = soup.find_all('div', {'class':'style__product-description___2XaG0'})
names = []
for i in n:
  names.append(i.text)

In [4]:
p = soup.find_all('div',{'class':'style__price-tag___cOxYc'})
prices = []
for i in p:
  prices.append(i)

In [5]:
r = soup.find_all('div',{'class':'CardRatingDetail__ratings-container___2ZTSK'})
ratings = []
for i in r:
  ratings.append(i.text)

In [6]:
print(len(names))
print(len(prices))
print(len(ratings))

0
0
0


doing everything at one shot

In [7]:
products = soup.find_all('div',{'class':'style__product-box___liepi'})
names = []
prices = []
ratings = []
try:
  for product in products:
    names.append(product.find_all('div', {'class':'style__product-description___2XaG0'})[0].text)
    prices.append(product.find_all('div',{'class':'style__price-tag___cOxYc'})[0].text)
    ratings.append(soup.find_all('div',{'class':'CardRatingDetail__ratings-container___2ZTSK'})[0].text)
except:
  ratings.append(0)

In [8]:
print(len(names))
print(len(prices))
print(len(ratings))
dic = {'Product Names':names, 'Prices':prices,'Ratings':ratings}

0
0
0


In [9]:
df = pd.DataFrame(dic)
df.to_csv('output.csv')

multipage

In [16]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape and parse data from a single page
def scrape_amazon_page(url):
    headers = {
        'User-Agent': 'Your User Agent Here',
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        book_info_containers = soup.find_all('div', class_='s-result-item')

        data = []

        for container in book_info_containers:
            book_name = container.find('span', class_='a-size-medium')
            author_name = container.find('a', class_='a-size-base')
            price = container.find('span', class_='a-price-whole')
            review_count = container.find('span', class_='a-size-base')

            # Check if the elements exist before extracting data
            if book_name and author_name and price and review_count:
                book_name = book_name.get_text()
                author_name = author_name.get_text()
                price = price.get_text()
                review_count = review_count.get_text()

                data.append([book_name, author_name, price, review_count])

        return data
    else:
        print("Failed to retrieve the page.")
        return None

# Main function to scrape data from multiple pages and store in a CSV file
def scrape_and_store_amazon_data(url, num_pages, output_file):
    all_data = []

    for page_num in range(1, num_pages + 1):
        page_url = f"{url}&page={page_num}"
        page_data = scrape_amazon_page(page_url)

        if page_data:
            all_data.extend(page_data)

    if all_data:
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Book Name', 'Author Name', 'Price', 'Review Count'])
            csv_writer.writerows(all_data)
        print(f"Data scraped from {num_pages} pages and stored in {output_file}.")
    else:
        print("No data to store.")

# Set the URL, number of pages to scrape, and output file name
url = 'https://www.amazon.in/novels/s?k=novels'
num_pages = 5  # You can adjust this to scrape more or fewer pages
output_file = 'amazon_novels.csv'

# Call the main scraping function
scrape_and_store_amazon_data(url, num_pages, output_file)


Data scraped from 5 pages and stored in amazon_novels.csv.
